<a href="https://colab.research.google.com/github/IFuentesSR/Soil_moisture/blob/main/SM_inputs4model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Import, authenticate and initialize the Earth Engine library.
import ee
ee.Authenticate()
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://accounts.google.com/o/oauth2/auth?client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code&code_challenge=mxh0BPY-lidF3nLesX4Ea5-m6lsXdRQ0h2ZlaB2hE7c&code_challenge_method=S256

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/1AX4XfWgQBirMu9aOz3LeU_ZfZ7Z_t7ksMjHPS5PFSpRGErt3iHHm2T-NOUE

Successfully saved authorization token.


In [2]:
import folium

## Preprocessing functions to stack predictors

In [16]:
def toNatural(img):
    '''
    Description: function that converts dB to natural values for S1 image
    Inputs: 
        img: ee.Image
    Outputs:
        ee.Image
    '''
    img = ee.Image(img)
    props = img.propertyNames()
    return ee.Image(10.0).pow(img.select(0).divide(10.0)).copyProperties(img, props)


def force_time_end(img):
    '''
    Description: set time_end in images
    Input: 
        img: ee.Image
    Output:
        ee.Image
    '''
    img = ee.Image(img)
    properties = img.propertyNames()
    condition = ee.Algorithms.If(properties.contains('system:time_end'),
                                 img,
                                 img.set('system:time_end', img.get('system:time_start')))
    return ee.Image(condition)


def toDB(img):
    '''
    Description: function that converts to dB values for S1 image
    Input: 
        img: ee.Image
    Output:
        ee.Image
    '''
    img = ee.Image(img)
    properties = img.propertyNames()
    return img.log10().multiply(10.0).copyProperties(img, properties)


# DOI: 10.1109/TGRS.2018.2858004
def edgeRemoval(img):
    '''
    Description: clamp sentinel 1 db values between -20 and -5
    Input: 
        img: ee.Image
    Output:
        ee.Image
    '''
    img = ee.Image(img)
    return img.updateMask(img.gt(-20).And(img.lt(-5)))


def gaussianFilter(img):
    '''
    Description: applies a 3x3 kernel gaussian filter to image
    Input: 
        img: ee.Image
    Output:
        ee.Image
    '''
    properties = img.propertyNames()
    img = ee.Image(resampleImage(ee.Image(edgeRemoval(img))))
    return img.convolve(ee.Kernel.gaussian(3, 3)).copyProperties(img,
                                                                 properties)
    

def angleCorrection2(slope):
    '''
    Description: applies correction to S1 image based on incidence angle
    Input: 
        img: ee.Image
    Output:
        ee.Image
    '''
    def inner(img):
        img = ee.Image(img)
        props = img.propertyNames()
        ols = img.addBands(img.select('VV').subtract(ee.Image(slope).multiply(img.select('angle2').subtract(30))).rename('sigma30'))
        return ols.copyProperties(img, props)
    return inner


def resampleAngle(img):
    '''
    Description: Bicubic resampling of incidence angle band in S1 images
    Input: 
        img: ee.Image
    Output:
        ee.Image
    '''
    img = ee.Image(img)
    props = img.propertyNames()
    resampled = img.addBands(img.select('angle').resample('bicubic').rename('angle2'))
    return resampled.copyProperties(img, props)


def rSSM(minimum, maximum):
    '''
    Description: calculates the rSSM based on normalization with 2 extremes
    Input: 
        img: ee.Image
        minimum: ee.Image corresponding to 5 percentil in sigma30 collection
        maximum: ee.Image corresponding to 95 percentil in sigma30 collection
    Output:
        ee.Image
    '''
    def inner(img):
        img = ee.Image(img)
        props = img.propertyNames()
        img1 = img.where(img.select('sigma30').lt(minimum), minimum)
        img1 = img1.select('sigma30').where(img1.select('sigma30').gt(maximum), maximum)
        rSSM = img1.select('sigma30').subtract(minimum).divide(maximum.subtract(minimum)).rename('rSSM')
        return img.select('VV').addBands(rSSM).copyProperties(img, props)
    return inner


def resampleImage(img):
    '''
    Description: resamples S1 image to lower resolution (90 m)
    Input: 
        img: ee.Image
    Output:
        ee.Image
    '''
    img = ee.Image(img)
    props = img.propertyNames()
    return img.select('VV').reduceResolution('mean',
                                             False,
                                             102).reproject('EPSG:4326',
                                                            None,
                                                            90).copyProperties(img, props)
                                                                            

def pre_process(img):
    '''
    Description: preprocess images for sigma30 collection generation
    Input: 
        img: ee.Image
    Output:
        ee.Image
    '''
    img = ee.Image(img)
    properties = img.propertyNames()
    resampled = ee.Image(gaussianFilter(ee.Image(toDB(img.select('VV')))))
    angle = ee.Image(resampleAngle(img.select('angle'))).select('angle2').reproject('EPSG:4326', None, 90)
    return ee.Image(resampled).addBands(angle).copyProperties(img, properties)


def get_rssm(geometry):
    '''
    Description: Creates collection of rSSM 
    Input: 
        geometry: ee.Geometry
    Output:
        ee.ImageCollection
    '''
    coll = S1.filterDate(start, end).filterMetadata('instrumentMode', "equals", 'IW').filterBounds(geometry)
    coll = coll.filterMetadata('orbitProperties_pass', 'equals', 'DESCENDING')
    coll = coll.filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV'))
    coll = coll.select(['VV', 'angle']).map(force_time_end)
    preColl = coll.map(pre_process)
    LinearReducer = ee.Reducer.linearFit()
    linear2 = ee.ImageCollection(preColl).select(['angle2', 'VV']).reduce(LinearReducer)
    beta = linear2.select('scale')
    sigma30 = preColl.map(angleCorrection2(beta))
    percentiles = sigma30.reduce(ee.Reducer.percentile([5, 95], ['min', 'max']))
    min = percentiles.select('sigma30_min')
    max = percentiles.select('sigma30_max')
    soilMoi = sigma30.map(rSSM(min, max))
    return soilMoi


def attach_chirps(img):
    '''
    Description: Aggregates CHIRPS rainfall between dates
    Input: 
        img: ee.Image
    Output:
        ee.Image
    '''
    date0 = ee.Date(img.get('system:time_start'))
    date1 = ee.Date(img.get('system:time_end'))
    chirp = chirps.filterDate(date0, date1).sum()
    return img.set('chirp', chirp)


def cat_modis(img):
    '''
    Description: Concatenates MODIS inputs as a multiband image
    Input: 
        img: ee.Image
    Output:
        ee.Image
    '''
    img = ee.Image(img)
    props = img.propertyNames()
    sr = img.select(['sur_refl_b01', 'sur_refl_b02', 'sur_refl_b03',
                     'sur_refl_b04', 'sur_refl_b05', 'sur_refl_b06',
                     'sur_refl_b07']).multiply(0.0001)
    ndvi = sr.normalizedDifference(['sur_refl_b02', 'sur_refl_b01']).rename('NDVI')
    lst = ee.Image(ee.List(img.get('lst')).get(0))
    lst = lst.select(['LST_Day_1km', "LST_Night_1km", "Emis_31", "Emis_32"])
    lst = lst.multiply(ee.Image.cat([ee.Image(0.02), ee.Image(0.02), ee.Image(0.002), ee.Image(0.002)]))
    retrieve = sr.addBands(lst).addBands(ndvi).select(modis_bands)
    return retrieve.copyProperties(img, props)


def wrap(img):
    '''
    Description: retrieves CHIRPS image from properties
    Input: 
        img: ee.Image
    Output:
        ee.Image
    '''
    return ee.Image(img.get('chirp'))


def get_image(img):
    '''
    Description: Stacks all bands together for the predictor images
    Input: 
        img: ee.Image
    Output:
        ee.Image
    '''
    img =  ee.Image(img)
    smap = ee.ImageCollection.fromImages(ee.List(img.get('images')).slice(0,10)).toBands().rename(smap_bands)
    chirp = ee.ImageCollection.fromImages(ee.List(img.get('images')).slice(0,10)).map(wrap).toBands().rename(chirps_bands)
    modis = ee.Image(ee.List(img.get('modis')).get(0)).select(modis_bands)
    sampleImg = img.addBands(modis).addBands(Merit).addBands([CLY, AWC, SOC]).addBands(smap).addBands(chirp).addBands(LUx).addBands(elevation)
    return sampleImg


def inner2(fea):
    '''
    Description: samples predictor image based on feature
    Input: 
        fea: ee.Feature
    Output:
        ee.Feature
    '''
    def inner(img):
        sampleImg = ee.Image(get_image(img))
        container = ee.Feature(ee.FeatureCollection(sampleImg.sample(region=fea.geometry(),
                                                                     scale=90,
                                                                     dropNulls=False)).first())
        container = container.setMulti({'date': ee.Date(img.get('system:time_start')).format('YYYY-MM-dd'),
                                        'id': fea.get('ID'),
                                        'site':fea.get('site')})
        return container
    return inner


def combining_datasets(fea):
    '''
    Description: Combines collections based on feature
    Input: 
        fea: ee.Feature
    Output:
        ee.ImageCollection
    '''
    s1_collection = get_rssm(fea.geometry())
    filter = ee.Filter.And(ee.Filter.maxDifference(difference=35*1000*86400,
                                                   leftField=timeField,
                                                   rightField=timeField),
                           ee.Filter.greaterThan(leftField=timeField,
                                                 rightField=timeField))
    joined = ee.Join.saveAll(matchesKey='images',
                             measureKey='SMAP',
                             ordering=timeField,
                             ascending=False).apply(primary=s1_collection,
                                                    secondary=SMAP,
                                                    condition=filter)
    filter2 = ee.Filter.And(ee.Filter.greaterThanOrEquals(leftField=timeField,
                                                          rightField=timeField),
                            ee.Filter.lessThanOrEquals(leftField='system:time_end',
                                                       rightField='system:time_end'))
    joined = ee.Join.saveAll(matchesKey='modis',
                             ordering=timeField,
                             ascending=True).apply(primary=joined,
                                                   secondary=Modis,
                                                   condition=filter2)
    return joined


def to_drive(fea):
    '''
    Description: Samples Collection based on feature
    Input: 
        fea: ee.Feature
    Output:
        ee.FeatureCollection
    '''
    joined = combining_datasets(fea)
    retrieve = joined.map(inner2(fea), True)
    return retrieve


def get_collection(fea):
    '''
    Description: Creates predictor ImageCollection based on feature
    Input: 
        fea: ee.Feature
    Output:
        ee.ImageCollection
    '''    
    joined = combining_datasets(fea)
    collection = joined.map(get_image)
    return collection


# Using entire S1 image leads to memory errors, but this is the function to get geometry from images
def get_geometry(img):
    '''
    Description: Returns the countour of Sentinel images as Feature
    Input: 
        img: ee.Image
    Output:
        ee.Feature
    '''
    contour = ee.Geometry(ee.Image(img).get('system:footprint')).coordinates()
    return ee.Feature(ee.Geometry.Polygon(contour))


def to_yyyymmdd(i):
    '''
    Description: Formats dates into %Y-%m-%d
    Input: 
        i: string or ee.Date
    Output:
        string
    '''
    return ee.Date(i).format('YYYY-MM-dd')


## Parameters definition

In [4]:
start = '2016-01-01'
end = '2021-05-01'
timeField = 'system:time_start'

smap_bands = ['smap1', 'ssmap1', 'smap2', 'ssmap2', 'smap3', 'ssmap3',
              'smap4', 'ssmap4', 'smap5', 'ssmap5', 'smap6', 'ssmap6',
              'smap7', 'ssmap7', 'smap8', 'ssmap8', 'smap9', 'ssmap9',
              'smap10', 'ssmap10']

chirps_bands = ['chirps1', 'chirps2', 'chirps3', 'chirps4', 'chirps5', 
                'chirps6', 'chirps7', 'chirps8', 'chirps9', 'chirps10']

modis_bands = ['LST_Day_1km', "LST_Night_1km", "Emis_31", "Emis_32",
               'NDVI', 'sur_refl_b01', 'sur_refl_b02', 'sur_refl_b03',
               'sur_refl_b04', 'sur_refl_b05', 'sur_refl_b06', 'sur_refl_b07']

timeFilter = ee.Filter.equals(leftField=timeField,
                              rightField=timeField)

saveAllJoin = ee.Join.saveAll(matchesKey='lst',
                              ordering=timeField,
                              ascending=True)

## Loading of Inputs

In [5]:
S1 = ee.ImageCollection('COPERNICUS/S1_GRD_FLOAT')
SMAP = ee.ImageCollection("NASA_USDA/HSL/SMAP10KM_soil_moisture").select(['ssm', 'susm']).filterDate('2000-01-01', '2021-11-29')
AWC = ee.Image('CSIRO/SLGA/AWC').select(['AWC_000_005_EV', 'AWC_005_015_EV', 'AWC_015_030_EV', 'AWC_030_060_EV']).rename(['AWC1', 'AWC2', 'AWC3', 'AWC4'])
CLY = ee.Image('CSIRO/SLGA/CLY').select(['CLY_000_005_EV', 'CLY_005_015_EV', 'CLY_015_030_EV', 'CLY_030_060_EV']).rename(['CLY1', 'CLY2', 'CLY3', 'CLY4'])
SOC = ee.Image('CSIRO/SLGA/SOC').select(['SOC_000_005_EV', 'SOC_005_015_EV', 'SOC_015_030_EV', 'SOC_030_060_EV']).rename(['SOC1', 'SOC2', 'SOC3', 'SOC4'])
Merit = ee.Image("MERIT/Hydro/v1_0_1").select(['upa', 'hnd'])
LST = ee.ImageCollection('MODIS/006/MOD11A2')
SR = ee.ImageCollection('MODIS/006/MOD09A1')
chirps = ee.ImageCollection("UCSB-CHG/CHIRPS/DAILY")
LU = ee.Image("users/ignaciofuentessanroman/AU/clum_50m1218m")
MCD = ee.ImageCollection("MODIS/006/MCD12Q1").select('LC_Type1').filterDate('2017-01-01', '2021-01-01').reduce(ee.Reducer.mode())
elevation = ee.Image("CGIAR/SRTM90_V4")
slope = ee.Terrain.slope(elevation).rename('slope')
aspect = ee.Terrain.aspect(elevation).rename('aspect')
elevation = elevation.addBands([slope, aspect])

## Land cover standardisation

In [6]:
LUx = LU.where(LU.lte(210), 1) #pasture
LUx = LUx.where(LU.gt(210).And(LU.lte(314)), 2) # forest
LUx = LUx.where(LU.gt(319).And(LU.lte(325)), 1)
LUx = LUx.where(LU.gte(330).And(LU.lte(360)), 3) # agri
LUx = LUx.where(LU.gt(359).And(LU.lte(400)), 1) # 
LUx = LUx.where(LU.gte(410).And(LU.lt(420)), 4) # irri
LUx = LUx.where(LU.gte(420).And(LU.lt(430)), 4) # 
LUx = LUx.where(LU.gte(430).And(LU.lte(465)), 4)
LUx = LUx.where(LU.gte(510).And(LU.lt(600)), 1) 
LUx = LUx.where(LU.gt(600).And(LU.lt(650)), 1)
LUx = LUx.where(LU.gte(650), 1)
LUx = LUx.where(LUx.eq(1).And(MCD.lte(5)), 2)
LUx = LUx.where(LUx.eq(1).And(MCD.gte(5)).And(MCD.lte(6)), 5)
LUx = LUx.where(LUx.eq(1).And(MCD.gte(7)).And(MCD.lte(9)), 6)
LUx = LUx.rename('LU')

In [7]:
irri = LUx.eq(4)
past = LUx.eq(1)
agri = LUx.eq(3)
fore = LUx.eq(2)
sava = LUx.gte(5)
LUx = ee.Image.cat([irri.rename('irri'), past.rename('past'), agri.rename('agri'),
                    fore.rename('fore'), sava.rename('sava')])

## Parameters to change for image selection

In [8]:
geo = ee.Geometry.Point([149.2659, -30.0996])
date = '2019-12-15'

## Getting Collection of predictor images based on geometry

In [9]:
SMAP = SMAP.map(attach_chirps)
Modis = saveAllJoin.apply(SR, LST, timeFilter)
Modis = ee.ImageCollection(Modis.map(cat_modis))
collection = get_collection(ee.Feature(geo))

In [18]:
print(collection.aggregate_array('system:time_start').map(to_yyyymmdd).getInfo())

['2016-01-07', '2016-01-19', '2016-01-31', '2016-01-31', '2016-02-12', '2016-02-24', '2016-03-07', '2016-03-19', '2016-03-31', '2016-04-12', '2016-04-24', '2016-05-06', '2016-05-18', '2016-05-30', '2016-06-11', '2016-07-05', '2016-07-17', '2016-07-29', '2016-08-10', '2016-08-22', '2016-09-03', '2016-09-15', '2016-09-27', '2016-10-09', '2016-10-21', '2016-11-02', '2016-11-14', '2016-11-26', '2016-12-08', '2016-12-20', '2017-01-01', '2017-01-13', '2017-01-25', '2017-02-06', '2017-02-18', '2017-03-02', '2017-03-14', '2017-03-26', '2017-04-07', '2017-04-19', '2017-05-01', '2017-05-13', '2017-05-25', '2017-06-06', '2017-06-18', '2017-06-30', '2017-07-12', '2017-07-24', '2017-08-05', '2017-08-17', '2017-08-29', '2017-09-10', '2017-09-22', '2017-10-04', '2017-10-16', '2017-10-28', '2017-11-09', '2017-11-21', '2017-12-03', '2017-12-15', '2017-12-27', '2018-01-08', '2018-01-20', '2018-02-01', '2018-02-13', '2018-02-25', '2018-03-09', '2018-03-21', '2018-04-02', '2018-04-14', '2018-04-26', '2018

## Setting band names

In [10]:
ixs = range(1, 11)
smap_cols = ['smap{}'.format(str(n)) for n in ixs]
ssmap_cols = ['ssmap{}'.format(str(n)) for n in ixs]
chirps_cols = ['chirps{}'.format(str(n)) for n in ixs]

lstm_cols = smap_cols+chirps_cols+ssmap_cols

#removed hnd, upa, emissivity
mlp_cols = ['VV', 'rSSM',
            'AWC1', 'AWC2', 'AWC3', 'AWC4',
            'CLY1', 'CLY2', 'CLY3', 'CLY4',
            'SOC1', 'SOC2', 'SOC3', 'SOC4',
            'LST_Day_1km', 'LST_Night_1km',
            'sur_refl_b01', 'sur_refl_b02', 'sur_refl_b03',
            'sur_refl_b04', 'sur_refl_b05', 'sur_refl_b06', 'sur_refl_b07',
            'NDVI', 'irri', 'past', 'agri', 'fore', 'sava'] 

bands = lstm_cols + mlp_cols

## Input image

In [11]:
img = ee.Image(collection.filterDate(date, '2022-01-01').first()).select(bands)

## Plotting some input bands and geometry

In [12]:
map0 = ee.Image(img).select('VV').getMapId({'min':-20, 'max':-5, })
mapid = ee.Image(img).select('rSSM').getMapId({'min':0, 'max':1, 'palette':'FF0000, FFFF00, 00FF00, 00FFFF, 0000FF'})
map = folium.Map(location=ee.Feature(geo).geometry().coordinates().getInfo()[::-1], zoom_start=10)
folium.TileLayer(
    tiles = 'https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
    attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    overlay=True,
    name='satellite',
  ).add_to(map)
folium.TileLayer(
    tiles=map0['tile_fetcher'].url_format,
    attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    overlay=True,
    name='VV',
  ).add_to(map)
folium.TileLayer(
    tiles=mapid['tile_fetcher'].url_format,
    attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    overlay=True,
    name='rssm',
  ).add_to(map)
map.add_child(folium.LayerControl())
map

In [15]:
mapid = ee.Feature(get_geometry(img)).getMapId()
map = folium.Map(location=ee.Feature(geo).geometry().coordinates().getInfo()[::-1], zoom_start=10)
folium.TileLayer(
    tiles = 'https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
    attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    overlay=True,
    name='satellite',
  ).add_to(map)
folium.TileLayer(
    tiles=mapid['tile_fetcher'].url_format,
    attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    overlay=True,
    name='geometry',
  ).add_to(map)
map.add_child(folium.LayerControl())
map

## Exporting predictor image

In [ ]:
date = ee.Date(img.get('system:time_start')).format('YYYY_MM_dd').getInfo()
geo_exp = ee.Geometry.Polygon([[[149.13340337807065, -30.108940652709176],
                                [149.13340337807065, -30.516764241304145],
                                [149.72117193275815, -30.516764241304145],
                                [149.72117193275815, -30.108940652709176]]], None, False)
ee.batch.Export.image.toDrive(img.toFloat().clip(geo_exp),
                              'moist_input_{}'.format(date),
                              region=geo_exp,
                              scale=90,
                              maxPixels=1e13).start()
